### Dataset

#### Allen Institute for Brain Science: Visual Coding - Neuropixels

© 2015 Allen Institute for Brain Science. Allen Brain Atlas API. Available from: https://portal.brain-map.org/explore/circuits/visual-coding-neuropixels).

## Basic dataset details
e.g. number of subjects, genotypes, and sexes

##### Imports

In [1]:
# imports
from os.path import join

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

##### Load session data

In [4]:
# set manifest path and project cache
manifest_path = join('/micha/visual_encoding/ecephys_cache_dir/', "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

# get session info for Brain Observatory sessions
sessions_all = cache.get_session_table()
sessions = sessions_all[sessions_all["session_type"] == "brain_observatory_1.1"] # "functional_connectivity"] # 
session_ids = sessions.index

# print number of sessions
print('Number of sessions (subjects): %d' %len(sessions))

Number of sessions (subjects): 32


#### Sex and genotype

In [5]:
for gt in np.unique(sessions['full_genotype']):
    n = np.sum(sessions['full_genotype']==gt)
    nM = np.sum([(sessions['full_genotype']==gt) & (sessions['sex']=='M')])
    nF = np.sum([(sessions['full_genotype']==gt) & (sessions['sex']=='F')])
    
    print('---------------------------------------------------------')
    print('Genotype: %s' %gt)
    print('Subjects: %d (%d females)' %(n, nF))


---------------------------------------------------------
Genotype: Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt
Subjects: 5 (2 females)
---------------------------------------------------------
Genotype: Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt
Subjects: 6 (1 females)
---------------------------------------------------------
Genotype: Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt
Subjects: 5 (2 females)
---------------------------------------------------------
Genotype: wt/wt
Subjects: 16 (0 females)
